# Modeling with NN 

In [1]:
## Import need packages 
from keras.wrappers.scikit_learn import KerasClassifier
import pandas as pd
import numpy as np

from keras.models import Sequential
from keras.layers import Dense, BatchNormalization,Dropout

from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [6]:
def print_scores(model):
    print(f'train score: {model.score(X_train, y_train)}')
    print(f'test score: {model.score(X_test, y_test)}')

In [2]:
def create_model(units=60):
    model = Sequential()
    model.add(Dense(12, input_shape=(750,), activation='relu'))
    model.add(BatchNormalization())
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(.4))
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(.3))
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(.2))
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(.2))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [7]:
df_reviews = pd.read_csv('../data/Las_Vegas_400_reviews.csv')
df_revs_combined = df_reviews.groupby(['business_id', 'name', 
                               'address', 'city' ,
                               'state', 'postal_code', 
                               'latitude' ,'longitude' , 
                               'stars', 'review_count', 
                               'is_open', 'attributes', 'categories']).agg({'text': ' '.join})
df_revs_combined = df_revs_combined.reset_index()
df_revs_combined['review_wc'] = df_revs_combined['text'].str.split().str.len()

X = df_revs_combined['text']
y = df_revs_combined['is_open']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=251)

In [11]:
tf_vect = TfidfVectorizer(stop_words='english', max_features=750)
X_train_trans = tf_vect.fit_transform(X_train)
X_test_trans = tf_vect.transform(X_test)
X_train_trans = X_train_trans.toarray()
X_test_trans = X_test_trans.toarray()

In [20]:
X_train_trans.shape, X_test_trans.shape

((297, 750), (100, 750))

In [13]:
sscaler = StandardScaler(with_mean=False)
model = KerasClassifier(build_fn=create_model, epochs = 10)
pipe = make_pipeline(sscaler,model)

In [14]:
pipe.fit(X_train_trans, y_train)

Epoch 1/10
10/10 [==============================] - 1s 2ms/step - loss: 0.6807 - accuracy: 0.5174
Epoch 2/10
10/10 [==============================] - 0s 3ms/step - loss: 0.6421 - accuracy: 0.6323
Epoch 3/10
10/10 [==============================] - 0s 2ms/step - loss: 0.6057 - accuracy: 0.7096
Epoch 4/10
10/10 [==============================] - 0s 3ms/step - loss: 0.5518 - accuracy: 0.7264
Epoch 5/10
10/10 [==============================] - 0s 2ms/step - loss: 0.4636 - accuracy: 0.8208
Epoch 6/10
10/10 [==============================] - 0s 3ms/step - loss: 0.3958 - accuracy: 0.8066
Epoch 7/10
10/10 [==============================] - 0s 3ms/step - loss: 0.3261 - accuracy: 0.8611
Epoch 8/10
10/10 [==============================] - 0s 2ms/step - loss: 0.2827 - accuracy: 0.8839
Epoch 9/10
10/10 [==============================] - 0s 3ms/step - loss: 0.1835 - accuracy: 0.9530
Epoch 10/10
10/10 [==============================] - 0s 3ms/step - loss: 0.1684 - accuracy: 0.9417


Pipeline(steps=[('standardscaler', StandardScaler(with_mean=False)),
                ('kerasclassifier',
                 <tensorflow.python.keras.wrappers.scikit_learn.KerasClassifier object at 0x7fac104831c0>)])

In [15]:
params = {'kerasclassifier__units': [10, 20, 100]}
grid = GridSearchCV(pipe, param_grid=params)

In [16]:
grid.fit(X_train_trans, y_train)

Epoch 1/10
8/8 [==============================] - 0s 2ms/step - loss: 0.6944 - accuracy: 0.4741
Epoch 2/10
8/8 [==============================] - 0s 3ms/step - loss: 0.6761 - accuracy: 0.5362
Epoch 3/10
8/8 [==============================] - 0s 3ms/step - loss: 0.6425 - accuracy: 0.6078
Epoch 4/10
8/8 [==============================] - 0s 3ms/step - loss: 0.6057 - accuracy: 0.7084
Epoch 5/10
8/8 [==============================] - 0s 2ms/step - loss: 0.5398 - accuracy: 0.7285
Epoch 6/10
8/8 [==============================] - 0s 3ms/step - loss: 0.4658 - accuracy: 0.7896
Epoch 7/10
8/8 [==============================] - 0s 3ms/step - loss: 0.2975 - accuracy: 0.9070
Epoch 8/10
8/8 [==============================] - 0s 2ms/step - loss: 0.2650 - accuracy: 0.9087
Epoch 9/10
8/8 [==============================] - 0s 2ms/step - loss: 0.1848 - accuracy: 0.9231
Epoch 10/10
2/2 [==============================] - 0s 2ms/step - loss: 1.5365 - accuracy: 0.6333
Epoch 1/10
8/8 [=======================

GridSearchCV(estimator=Pipeline(steps=[('standardscaler',
                                        StandardScaler(with_mean=False)),
                                       ('kerasclassifier',
                                        <tensorflow.python.keras.wrappers.scikit_learn.KerasClassifier object at 0x7fac104831c0>)]),
             param_grid={'kerasclassifier__units': [10, 20, 100]})

In [17]:
grid.score(X_train_trans, y_train)

10/10 [==============================] - 0s 682us/step - loss: 0.2041 - accuracy: 0.9360


0.936026930809021

In [18]:
grid.score(X_test_trans, y_test)

4/4 [==============================] - 0s 876us/step - loss: 1.1364 - accuracy: 0.7200


0.7200000286102295

In [19]:
grid.best_params_

{'kerasclassifier__units': 20}